In [6]:
import roar_py_carla
import roar_py_interface
import carla
import numpy as np
from typing import List, Tuple
import transforms3d as tr3d
import copy

carla_client = carla.Client('localhost', 2000)
carla_client.set_timeout(15.0)
roar_py_instance = roar_py_carla.RoarPyCarlaInstance(carla_client)
roar_py_world = roar_py_instance.world
roar_py_world.set_asynchronous(True)
roar_py_world.set_control_steps(0.00, 0.005)

In [7]:
# ground waypoints
current_waypoints = roar_py_world.maneuverable_waypoints
grounded_waypoints : List[roar_py_interface.RoarPyWaypoint] = []

for waypoint in current_waypoints:
    new_waypoint = copy.deepcopy(waypoint)
    new_waypoint.location = roar_py_world.ground_projection(waypoint.location + np.array([0,0,2]), 100.0)
    assert new_waypoint.location is not None
    grounded_waypoints.append(new_waypoint)

In [9]:
# check rpy
def normalize_rad(r):
    return np.arctan2(np.sin(r), np.cos(r))

final_waypoints : List[roar_py_interface.RoarPyWaypoint] = []
for i,ground_waypoint in enumerate(grounded_waypoints):
    new_waypoint = copy.deepcopy(ground_waypoint)
    next_waypoint, prev_waypoint = grounded_waypoints[(i+1)%len(grounded_waypoints)], grounded_waypoints[(i-1)%len(grounded_waypoints)]
    center_yaw = np.arctan2(next_waypoint.location[1] - prev_waypoint.location[1], next_waypoint.location[0] - prev_waypoint.location[0])
    center_rot = np.array([0,0,center_yaw])
    if np.linalg.norm(ground_waypoint.roll_pitch_yaw - center_rot) > np.pi:
        print("Fixing angle", ground_waypoint.roll_pitch_yaw, center_rot)
        print(ground_waypoint, next_waypoint, prev_waypoint)
        new_waypoint.roll_pitch_yaw = center_rot
    final_waypoints.append(new_waypoint)

Fixing angle [ 5.26012107e-05 -1.78813919e-05 -3.13952178e+00] [0.         0.         3.12475681]
RoarPyWaypoint(location=array([-2.0628029e+02, -1.0638969e+03, -1.5258789e-07], dtype=float32), roll_pitch_yaw=array([ 5.26012107e-05, -1.78813919e-05, -3.13952178e+00]), lane_width=12.0) RoarPyWaypoint(location=array([ -208.3741, -1063.8259,     0.    ], dtype=float32), roll_pitch_yaw=array([-2.59498365e-03,  4.05311572e-05,  3.09273537e+00]), lane_width=12.0) RoarPyWaypoint(location=array([-2.0419087e+02, -1.0638964e+03,  1.5258789e-07], dtype=float32), roll_pitch_yaw=array([-1.32359080e-03,  1.12056721e-05, -3.13536618e+00]), lane_width=12.0)


In [10]:
# save

np.savez_compressed(
    "Monza.npz",
    **roar_py_interface.RoarPyWaypoint.save_waypoint_list(final_waypoints)
)